# Rozwiązania do zadań dla bloku Python Średniozaawansowany

## Wyjątki

1. Napisz klasę `AutodestructingFile` która stworzy plik o lokalizacji zadanej w konstruktorze i pozwoli na zapisywanie i odczytywanie z owego pliku. Obiekt tej klasy, gdy użyty w ramach klauzuli `with`, ma usunąć automatycznie plik z dysku w momencie zakończenia działania, niezależnie od tego, czy klauzula zakończyła się poprawnie czy błędem. Należy zastąpić wszystkie wystąpienia `pass` odpowiednim kodem.

##### Rozwiązanie

 * podpowiedź 1 - zapisz zmienną `address` w konstruktorze jako pole obiektu. Następnie stwórz pola `reader` i `writer` i przypisz do nich odpowiednio `open(address, "w")` i `open(address, "r")`.

 * podpowiedź 2 - w operacji `write` zapisz tekst do pliku odwołując się do pola `self.writer`. Analogicznie zczytaj treść pliku z pomocą `self.reader` w operacji `write`.

 * podpowiedź 3 - W operacji `__exit__` zamknij `self.writer` i `self.reader` oraz usuń plik z pomocą operacji `os.path.exists` i `os.remove`.

Wywołanie powinno zakończyć się tekstem *"This code should fail here and dummy.file should not exist"*.

In [1]:
class AutodestructingFile:

    def __init__(self, address) -> None:
        self.address = address
        self.writer = open(address, "w")
        self.reader = open(address, "r")

    def write(self, text):
        self.writer.write(text)

    def read(self):
        return self.reader.read()

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        self.reader.close()
        self.writer.close()
        if os.path.exists(self.address):
            os.remove(self.address)


dummy_file_name = "dummy.file"
some_dummy_message = "somedummymessage"

assert not os.path.exists(dummy_file_name)

with AutodestructingFile(dummy_file_name) as file:
    assert os.path.exists(dummy_file_name)
    file.write(some_dummy_message)
    print(file.read())

    raise Exception(f"This code should fail here and {dummy_file_name} should not exist")
    print("This message should not be displayed")

Exception: This code should fail here and dummy.file should not exist

## Programowanie obiektowe

2. Wczytaj plik za pomocą `reader`, przetwórz treść za pomocą `converter` i zapisz do pliku za pomocą `writer`. Stwórz klasę `LUFileConverter`, która będzie konwertowała zadany plik zmieniając małe litery w wielkie i vice versa. Stwórz klasę `DoubleFileConverter`, która wykona operacje `converter` dwukrotnie.

`convertFile` w `FileConverter` ma być metodą abstrakcyjną.
`reader` ma posiadać operację `read()`, która zczyta obiekt typu `str` z pewnego pliku.
`writer` ma posiadać operację `write()`, która zapisze do pewnego pliku zadany obiekt typu `str`.
`converter` ma przyjąć obiekt typu `str` i zwrócić obiekt typu `str`.

##### Rozwiązanie

 - Stwórz plik (`dummy.txt` lub jakikolwiek inny) z dowolną treścią (`Ala ma kota` lub cokolwiek innego).
 - Wczytaj treść tego pliku za pomocą operacji `open` i `read`.
 - Zapisz treść tego pliku za pomocą `open` i `write` do innego pliku (`temp.file` lub jakikolwiek inny).
 - Stwórz funkcję `toUpper`, która przyjmie obiekt typu `str` i zwróci ów tekst z wszystkimi małymi literami zamienionymi na wielkie.
 - Stwórz obiekt typu `FileConverter` i zapisz jako jego pola `reader`, `writer` oraz `toUpper`.
 - Zmień pole `convertFile` w `FileConverter` na metodę abstrakcyjną.

In [2]:
from abc import ABC, abstractmethod

def swapCase(text: str) -> str:
    return text.swapcase()

class FileConverter(ABC):
    def __init__(self, reader, writer, converter) -> None:
        self.reader = reader
        self.writer = writer
        self.converter = converter

    @abstractmethod
    def convertFile(self):
        pass

class LUFileConverter(FileConverter):
    def __init__(self, reader, writer) -> None:
        super().__init__(reader, writer, swapCase)

    def convertFile(self):
        content = self.reader.read()
        content = self.converter(content)
        self.writer.write(content)

class DoubleFileConverter(FileConverter):
    def convertFile(self):
        content = self.reader.read()
        content = self.converter(content)
        content = self.converter(content)
        self.writer.write(content)

In [3]:
def doubleText(text):
    return text + text

in_file = "dummy.txt"
out_file = "temp.file"

with open(in_file, "w") as writer:
    writer.write("Ala ma kota\nA sierotka ma rysia\n\n")

with open(in_file, "r") as reader:
    with open(out_file, "w") as writer:
        fc = DoubleFileConverter(reader, writer, doubleText)
        fc.convertFile()

if os.path.exists(in_file):
    os.remove(in_file)
if os.path.exists(out_file):
    os.remove(out_file)

In [4]:
def doubleText(text):
    return text + text

in_file = "dummy.txt"
out_file = "temp.file"

with open(in_file, "w") as writer:
    writer.write("Ala ma kota\nA sierotka ma rysia\n\n")

with open(in_file, "r") as reader:
    with open(out_file, "w") as writer:
        fc = LUFileConverter(reader, writer)
        fc.convertFile()

if os.path.exists(in_file):
    os.remove(in_file)
if os.path.exists(out_file):
    os.remove(out_file)

3. Napisz strukturę o trzech niemutowalnych polach, którą można posortować oraz przechowywać jako element zbioru.

In [5]:
from dataclasses import dataclass

@dataclass(frozen=True, order=True, unsafe_hash=True)
class Triple:
    a: float
    b: float
    c: float

print(sorted([Triple(1, 2, 3), Triple(2, 1, 19), Triple(1, 2, 1), Triple(1, 3, 2)]))
print({Triple(1, 2, 3), Triple(2, 1, 19), Triple(1, 2, 1), Triple(1, 3, 2)})

[Triple(a=1, b=2, c=1), Triple(a=1, b=2, c=3), Triple(a=1, b=3, c=2), Triple(a=2, b=1, c=19)]
{Triple(a=1, b=2, c=1), Triple(a=1, b=3, c=2), Triple(a=1, b=2, c=3), Triple(a=2, b=1, c=19)}


## Serializacja

## Dekoratory

#### Napisz dekorator, który przechwyci wyjątek wyrzucany przez funkcję i wypisze jego treść i ślad na wyjściu konsoli (domyślnie wyjątki wypisują się na `stderr`, standardowe wyjście konsoli to `stdout`).

Do zadania wykorzystaj pakiet [traceback](https://docs.python.org/3/library/traceback.html).

In [6]:
import traceback, sys

def captureError(func):
    def wrapper():
        try:
            func()
        except Exception as e:
            traceback.print_exc(file=sys.stdout)
    return wrapper


@captureError
def function1():
    def function2():
        raise AssertionError("Why index error? Just so")
    print("First print")
    function2()
    print("Second print")

function1()

First print
Traceback (most recent call last):
  File "C:\Users\micha\AppData\Local\Temp/ipykernel_17192/2643493766.py", line 6, in wrapper
    func()
  File "C:\Users\micha\AppData\Local\Temp/ipykernel_17192/2643493766.py", line 17, in function1
    function2()
  File "C:\Users\micha\AppData\Local\Temp/ipykernel_17192/2643493766.py", line 15, in function2
    raise AssertionError("Why index error? Just so")
AssertionError: Why index error? Just so


#### Napisz dekorator `encrypted`, który przyjmuje jako parametry adres pliku oraz funkcję szyfrującą a następnie zapisuje do tego pliku zaszyfrowany wynik funkcji.

In [7]:
from random import shuffle
import string

basis = list(string.printable)
permutation = list(basis)
shuffle(permutation)

dictionary = {}
for i in range(len(basis)):
    dictionary[basis[i]] = permutation[i]

def encryptor(text):
    return ''.join([dictionary[c] for c in text])

In [8]:
dummy_file_name = "dummy.file"

def encrypted(encryptor, address):
    def dec(func):
        def wrapper():
            with open(address, "w") as writer:
                writer.write(encryptor(func()))
        return wrapper
    return dec

@encrypted(encryptor, dummy_file_name)
def strfunc():
    return "Ala ma kota"

strfunc()

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

#### Napisz powyższe zadanie, tym razem jednak dla funkcji, która przyjmuje argumenty

In [9]:
from random import shuffle
import string

basis = list(string.printable)
permutation = list(basis)
shuffle(permutation)

dictionary = {}
for i in range(len(basis)):
    dictionary[basis[i]] = permutation[i]

def encryptor(text):
    return ''.join([dictionary[c] for c in text])

In [10]:
dummy_file_name = "dummy.file"

def encrypted(encryptor, address):
    def dec(func):
        def wrapper(*args, **kwargs):
            with open(address, "w") as writer:
                writer.write(encryptor(func(*args, **kwargs)))
        return wrapper
    return dec

@encrypted(encryptor, dummy_file_name)
def strfunc(text, n):
    return f"My encrypted text is '{text}' and it should occure {n} times after this line\n" + "\n".join([text] * n)

strfunc("Ala ma kota", 5)

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)